# Peer-Graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 1: Extracting Borough & Neighborhood Table

In [1]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=3b743d8dcf0661933732581ef138adb252b3ca4ee09e3d13f124e8796700539d
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text

In [4]:
soup = BeautifulSoup(data, "html5lib")

In [5]:
table_contents = []
table = soup.find("table")

In [6]:
for row in table.findAll('td'):
    cell = {}
    if row.span.text == "Not assigned":
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = row.span.text.split('(')[0]
        cell['Neighborhood'] = row.span.text.split('(')[1].strip(')').replace(' /', ',').replace(')', ' ').strip(' ')
        table_contents.append(cell)

#### Cast the list of dictionaries into a dataframe and replace some longer borough names with shorter ones

In [7]:
df = pd.DataFrame(table_contents)
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                      'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [8]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


#### Check to see if every Postal Code has one and only one row assigned.

In [9]:
len(df['PostalCode'].unique()) == df.shape[0]

True

#### Check to see if there is a borough with 'Not assigned' neighborhood

In [10]:
for row in df.values:
    if 'Not assigned' in row[2]:
        print(row[2])
    else:
        break
print('Every Borough has at least one neighborhood assigned')

Every Borough has at least one neighborhood assigned


In [11]:
df.shape

(103, 3)


## Part 2: Extracting Coordinates for Neighborhoods

In [12]:
from geopy.geocoders import Nominatim

In [13]:
geolocator = Nominatim(user_agent='ontario_explorer')
# Create lists of postal code latitudes and longitudes to be added as columns to the dataframe
latitude_column = []
longitude_column = []
for index, row in df.iterrows():
    # Create lists of neighborhood latitudes and longitudes - averages of which will be postal latitudes and longitudes
    n_latitudes = []
    n_longitudes = []
    
    for neighborhood in row['Neighborhood'].split(','):
        try:
            address = '{}, Toronto, Ontario'.format(neighborhood)
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
        except:
            address = '{}, Toronto, Ontario'.format(row['Borough'])
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
        n_latitudes.append(latitude)
        n_longitudes.append(longitude)    
        
    # Postal coordinate is average of the neighborhood coordinates that the postal code contains
    postal_latitude = sum(n_latitudes)/len(n_latitudes)
    postal_longitude = sum(n_longitudes)/len(n_longitudes)
    print(row['PostalCode'], row['Neighborhood'], '-', row['Borough'], postal_latitude, postal_longitude)
    latitude_column.append(postal_latitude) 
    longitude_column.append(postal_longitude)

M3A Parkwoods - North York 43.7587999 -79.3201966
M4A Victoria Village - North York 43.732658 -79.3111892
M5A Regent Park, Harbourfront - Downtown Toronto 43.65039285 -79.3703032
M6A Lawrence Manor, Lawrence Heights - North York 43.7224286 -79.44421994999999
M7A Ontario Provincial Government - Queen's Park 43.659659 -79.3903399
M9A Islington Avenue - Etobicoke 43.6373968 -79.5204119
M1B Malvern, Rouge - Scarborough 43.80706295 -79.1937691
M3B Don Mills North - North York 43.775347 -79.3459439
M4B Parkview Hill, Woodbine Gardens - East York 43.7091881 -79.3122373
M5B Garden District, Ryerson - Downtown Toronto 43.65741195 -79.37801962843974
M6B Glencairn - North York 43.7087117 -79.4406853
M9B West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale - Etobicoke 43.64682306 -79.48284113614898
M1C Rouge Hill, Port Union, Highland Creek - Scarborough 43.78196406666666 -79.14627003333334
M3C Don Mills South - North York 43.775347 -79.3459439
M4C Woodbine Heights - East York 43

In [14]:
df['Latitude'] = latitude_column
df['Longitude'] = longitude_column
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650393,-79.370303
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722429,-79.444220
4,M7A,Queen's Park,Ontario Provincial Government,43.659659,-79.390340
5,M9A,Etobicoke,Islington Avenue,43.637397,-79.520412
6,M1B,Scarborough,"Malvern, Rouge",43.807063,-79.193769
7,M3B,North York,Don Mills North,43.775347,-79.345944
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.709188,-79.312237
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657412,-79.378020


## Part 3: Clustering the Neighborhoods of Toronto

In [15]:
import numpy as np
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.8-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    certifi-2021.5.30          |   py38h578d9bd_0         141 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.8             |           2_cp38           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    widgetsnbextension-3.5.1   |   py38h578d9bd_4         1.8 MB  conda-forge
    ------------------------------------------------------------
          

In [16]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent='on_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The coordinates of Toronto, Canada are {}, {}".format(latitude, longitude))

The coordinates of Toronto, Canada are 43.6534817, -79.3839347


### Create a map for all postal codes in Toronto

In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start = 10)
for index, rows in df.iterrows():
    label = '{} - {}: {}'.format(rows['PostalCode'], rows['Borough'], rows['Neighborhood'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [rows['Latitude'], rows['Longitude']],
    radius = 5,
    popup = label,
    color='blue',
    fill = True,
    fill_color='#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(map_toronto)
map_toronto

In [18]:
CLIENT_ID = 'M2CVJTZJM5P5RREB5R4YWBMMH2CGZDSCNU5NCCVBBZDE4RXM' 
CLIENT_SECRET = 'NJUV2YGSDJIFNSNAF4XGL1KEZU0XU2UWYBWIF2WD3CRYZTPS' 
VERSION = '20180605' 
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M2CVJTZJM5P5RREB5R4YWBMMH2CGZDSCNU5NCCVBBZDE4RXM
CLIENT_SECRET:NJUV2YGSDJIFNSNAF4XGL1KEZU0XU2UWYBWIF2WD3CRYZTPS


### Create a dataframe of venues of all postal codes within 1000 meters from the center

In [19]:
def getNearbyVenues(postalNames, boroughNames, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for pName, bName, lat, lng in zip(postalNames, boroughNames, latitudes, longitudes):
        print("{}: {}".format(pName, bName))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            pName,
            bName,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code',
                  'Borough',
                  'Postal Latitude', 
                  'Postal Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(df['PostalCode'], df['Borough'], df['Latitude'], df['Longitude'])

M3A: North York
M4A: North York
M5A: Downtown Toronto
M6A: North York
M7A: Queen's Park
M9A: Etobicoke
M1B: Scarborough
M3B: North York
M4B: East York
M5B: Downtown Toronto
M6B: North York
M9B: Etobicoke
M1C: Scarborough
M3C: North York
M4C: East York
M5C: Downtown Toronto
M6C: York
M9C: Etobicoke
M1E: Scarborough
M4E: East Toronto
M5E: Downtown Toronto
M6E: York
M1G: Scarborough
M4G: East York
M5G: Downtown Toronto
M6G: Downtown Toronto
M1H: Scarborough
M2H: North York
M3H: North York
M4H: East York
M5H: Downtown Toronto
M6H: West Toronto
M1J: Scarborough
M2J: North York
M3J: North York
M4J: East York/East Toronto
M5J: Downtown Toronto
M6J: West Toronto
M1K: Scarborough
M2K: North York
M3K: North York
M4K: East Toronto
M5K: Downtown Toronto
M6K: West Toronto
M1L: Scarborough
M2L: North York
M3L: North York
M4L: East Toronto
M5L: Downtown Toronto
M6L: North York
M9L: North York
M1M: Scarborough
M2M: North York
M3M: North York
M4M: East Toronto
M5M: North York
M6M: York
M9M: North York


In [21]:
toronto_venues.groupby('Postal Code').count()

,Borough,Postal Latitude,Postal Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,,
M1B,18,18,18,18,18,18,18
M1C,14,14,14,14,14,14,14
M1E,24,24,24,24,24,24,24
M1G,41,41,41,41,41,41,41
M1H,37,37,37,37,37,37,37
...,...,...,...,...,...,...,...
M9N,17,17,17,17,17,17,17
M9P,30,30,30,30,30,30,30
M9R,18,18,18,18,18,18,18


In [22]:
pd.set_option('display.max_columns', None)

### Create one hot encoding and carry out grouping to have a data frame for all postal codes mean fraction of venues 

In [23]:
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'], prefix= "", prefix_sep = "")
toronto_onehot['Postal Code'] = toronto_venues['Postal Code']
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped.head(20)

,Postal Code,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Sushi Res

### For every postal codes show the 10 most frequent venues

In [24]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pcode_venues_sorted = pd.DataFrame(columns = columns)
pcode_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    pcode_venues_sorted.iloc[ind, 1:] = toronto_grouped.iloc[ind, 1:].sort_values(ascending=False).index.values[0:num_top_venues]

pcode_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Trail,Fast Food Restaurant,Coffee Shop,Sandwich Place,Supermarket,Spa,Caribbean Restaurant,Gym,Chinese Restaurant,Paper / Office Supplies Store
1,M1C,Pizza Place,Playground,Gym,Pet Store,Coffee Shop,Bank,Liquor Store,Pharmacy,Beer Store,Food & Drink Shop
2,M1E,Pizza Place,Fast Food Restaurant,Coffee Shop,Bank,Restaurant,Supermarket,Laundromat,Beer Store,Thrift / Vintage Store,Greek Restaurant
3,M1G,Fast Food Restaurant,Department Store,Grocery Store,Coffee Shop,Sandwich Place,Park,Pizza Place,Discount Store,Czech Restaurant,Pharmacy
4,M1H,Fast Food Restaurant,Pizza Place,Grocery Store,Discount Store,Diner,Sandwich Place,Coffee Shop,Beer Store,Toy / Game Store,Thrift / Vintage Store


### Actual K-means Clustering starts here

In [25]:
kclusters = 4
toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)
print(len(kmeans.labels_))
kmeans.labels_

103


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 3, 1,
       1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 3, 1, 1, 1, 1, 0, 3, 1, 1, 1, 3, 3], dtype=int32)

In [26]:
#pcode_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)
pcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df.join(pcode_venues_sorted.set_index('Postal Code'), on = 'PostalCode')
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.758800,-79.320197,1,Coffee Shop,Gas Station,Discount Store,Supermarket,Pizza Place,Pharmacy,Bus Line,Fast Food Restaurant,Hakka Restaurant,Bowling Alley
1,M4A,North York,Victoria Village,43.732658,-79.311189,1,Park,Intersection,Thrift / Vintage Store,Pizza Place,Pet Store,Coffee Shop,Spa,Bus Line,Middle Eastern Restaurant,Mediterranean Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650393,-79.370303,1,Coffee Shop,Café,Restaurant,Park,Italian Restaurant,Theater,Hotel,Farmers Market,Gym,Japanese Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722429,-79.444220,1,Clothing Store,Park,Coffee Shop,Restaurant,Men's Store,Fast Food Restaurant,Dessert Shop,Food Court,Cosmetics Shop,Furniture / Home Store
4,M7A,Queen's Park,Ontario Provincial Government,43.659659,-79.390340,1,Coffee Shop,Japanese Restaurant,Park,Ramen Restaurant,Pizza Place,Gastropub,Vietnamese Restaurant,Plaza,Dessert Shop,Grocery Store


In [27]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0, 1] + list(range(6, toronto_merged.shape[1]))] ]

,PostalCode,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M9L,North York,Sports Bar,Coffee Shop,Mexican Restaurant,Adult Boutique,Optical Shop,Park,Paper / Office Supplies Store,Pakistani Restaurant,Outdoor Supply Store,Other Great Outdoors


### Cluster 2

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0, 1] + list(range(6, toronto_merged.shape[1]))]]

,PostalCode,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Coffee Shop,Gas Station,Discount Store,Supermarket,Pizza Place,Pharmacy,Bus Line,Fast Food Restaurant,Hakka Restaurant,Bowling Alley
1,M4A,North York,Park,Intersection,Thrift / Vintage Store,Pizza Place,Pet Store,Coffee Shop,Spa,Bus Line,Middle Eastern Restaurant,Mediterranean Restaurant
2,M5A,Downtown Toronto,Coffee Shop,Café,Restaurant,Park,Italian Restaurant,Theater,Hotel,Farmers Market,Gym,Japanese Restaurant
3,M6A,North York,Clothing Store,Park,Coffee Shop,Restaurant,Men's Store,Fast Food Restaurant,Dessert Shop,Food Court,Cosmetics Shop,Furniture / Home Store
4,M7A,Queen's Park,Coffee Shop,Japanese Restaurant,Park,Ramen Restaurant,Pizza Place,Gastropub,Vietnamese Restaurant,Plaza,Dessert Shop,Grocery Store
...,...,...,...,...,...,...,...,...,...,...,...,...
97,M5X,Downtown Toronto,Coffee Shop,Bakery,Vietnamese Restaurant,Café,Diner,Brewery,Ice Cream Shop,Pizza Place,Bar,French Restaurant
98,M8X,Etobicoke,Park,Coffee Shop,Breakfast Spot,Sushi Restaurant,Dessert Shop,Business Service,Italian Restaurant,Pub,Bank,Gym / Fitness Center
99,M4Y,Downtown Toronto,Coffee Shop,Japanese Restaurant,Park,Gay Bar,Sushi Restaurant,Men's Store,Sandwich Place,Thai Restaurant,Ramen Restaurant,Caribbean Restaurant
100,M7Y,East Toronto Business,Coffee Shop,Hotel,Café,Cosmetics Shop,Japanese Restaurant,Gym,Clothing Store,Gastropub,Restaurant,Seafood Restaurant


### Cluster 3

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0, 1] + list(range(6, toronto_merged.shape[1]))]]

,PostalCode,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,M1X,Scarborough,Zoo Exhibit,Park,Trail,Other Great Outdoors,Pizza Place,Gas Station,Fast Food Restaurant,Neighborhood,New American Restaurant,Nightclub


### Cluster 4

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0, 1] + list(range(6, toronto_merged.shape[1]))]]

,PostalCode,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M6B,North York,Grocery Store,Coffee Shop,Bakery,Park,Gym Pool,Japanese Restaurant,Shopping Mall,Bus Line,Gas Station,Restaurant
21,M6E,York,Furniture / Home Store,Park,Coffee Shop,Convenience Store,Gas Station,Grocery Store,Thrift / Vintage Store,Sandwich Place,Playground,Dessert Shop
33,M2J,North York,Coffee Shop,Intersection,Park,Trail,Convenience Store,Print Shop,Middle Eastern Restaurant,Tennis Court,Clothing Store,Fast Food Restaurant
34,M3J,North York,Coffee Shop,Theater,Pizza Place,Juice Bar,Park,Shopping Mall,Gas Station,Nightclub,Other Great Outdoors,Nail Salon
45,M2L,North York,Park,Coffee Shop,Bank,Supermarket,Ice Cream Shop,Shopping Mall,Bus Stop,Baseball Field,Sandwich Place,Cosmetics Shop
49,M6L,North York,Intersection,Gas Station,Park,Bakery,Convenience Store,Coffee Shop,Organic Grocery,Pastry Shop,Paper / Office Supplies Store,Pakistani Restaurant
56,M6M,York,Coffee Shop,Park,Furniture / Home Store,Playground,Fast Food Restaurant,Tennis Court,Grocery Store,Pizza Place,Intersection,Gas Station
57,M9M,North York,Park,Coffee Shop,Latin American Restaurant,Nightclub,Discount Store,Café,Grocery Store,Adult Boutique,Pastry Shop,Paper / Office Supplies Store
66,M2P,North York,Coffee Shop,Gym,Restaurant,Japanese Restaurant,Convenience Store,Thai Restaurant,Bus Station,Grocery Store,Sandwich Place,Golf Course
89,M9V,Etobicoke,Coffee Shop,Bus Station,Latin American Restaurant,Park,Hockey Arena,Grocery Store,Automotive Shop,Food & Drink Shop,Café,Discount Store
